In [1]:
# Importar librerías
import pandas as pd
import json
import joblib
import os

In [2]:
# Comparación de Modelos
models_dir = '../../models/'
model_comparison = {}

model_files = {
    "Linear Regression": "linear_regression",
    "Random Forest": "random_forest",
    "Gradient Boosting": "gradient_boosting",
    "XGBoost": "xgboost",
    "Random Forest Tuned": "random_forest_tuned",
    "Gradient Boosting Tuned": "gradient_boosting_tuned",
    "XGBoost Tuned": "xgboost_tuned"
}

for model_name, file_prefix in model_files.items():
    metrics_path = os.path.join(models_dir, f"{file_prefix}_metrics.json")
    pipeline_path = os.path.join(models_dir, f"{file_prefix}_pipeline.joblib")
    
    if os.path.exists(pipeline_path) and os.path.exists(metrics_path):
        with open(metrics_path, 'r') as f:
            metrics = json.load(f)
        
        r2_train = metrics.get('train_metrics', {}).get('r2')
        r2_test = metrics.get('test_metrics', {}).get('r2')
        
        # Heurística para detectar overfitting: diferencia de R² > 0.1
        overfitting_flag = (r2_train is not None and r2_test is not None) and (r2_train - r2_test > 0.1)

        model_comparison[model_name] = {
            "R² (Prueba)": r2_test,
            "RMSE (Prueba)": metrics.get('test_metrics', {}).get('rmse'),
            "MAE (Prueba)": metrics.get('test_metrics', {}).get('mae'),
            "R² (Entrenamiento)": r2_train,
            "RMSE (Entrenamiento)": metrics.get('train_metrics', {}).get('rmse'),
            "MAE (Entrenamiento)": metrics.get('train_metrics', {}).get('mae'),
            "Tiempo de entrenamiento (s)": metrics.get('training_time', 'N/A'),
            "Tiempo de predicción (s)": metrics.get('prediction_time', 'N/A'),
            "Posible Overfitting": "Sí" if overfitting_flag else "No"
        }

# Crear DataFrame para visualización
comparison_df = pd.DataFrame(model_comparison).T
comparison_df = comparison_df.sort_values(by="R² (Prueba)", ascending=False)

print("Comparación de Modelos:")
display(comparison_df)

Comparación de Modelos:


,R² (Prueba),RMSE (Prueba),MAE (Prueba),R² (Entrenamiento),RMSE (Entrenamiento),MAE (Entrenamiento),Tiempo de entrenamiento (s),Tiempo de predicción (s),Posible Overfitting
Random Forest Tuned,0.805242,12799.066542,7239.970685,0.948792,7042.678334,3574.780646,16.268839,0.027672,Sí
XGBoost Tuned,0.804433,12825.61392,7571.263868,0.926414,8442.440016,5520.776718,4.515549,0.003255,Sí
XGBoost,0.797995,13035.013988,7959.820767,0.893429,10159.896145,6591.142756,0.093971,0.002206,No
Gradient Boosting Tuned,0.79235,13215.911319,7760.551597,0.931173,8164.858567,5448.204019,14.036204,0.006338,Sí
Random Forest,0.781103,13569.087777,8169.500503,0.907552,9462.805343,6054.633599,0.500972,0.015496,Sí
Gradient Boosting,0.76972,13917.425139,8818.689028,0.825744,12991.65242,8349.936699,1.003435,0.00291,No
Linear Regression,0.470845,21097.08459,14326.372073,0.464384,22777.042507,14637.998718,0.011445,0.00149,No


In [3]:
# Decidir el modelo final
best_model_name = comparison_df.index[0]
print(f"\nEl modelo seleccionado por R² es: {best_model_name}")


El modelo seleccionado por R² es: Random Forest Tuned


In [4]:
best_model_name = comparison_df.sort_values('Tiempo de predicción (s)').index[0]
print(f"\nEl modelo seleccionado por tiempo de predicción es: {best_model_name}")


El modelo seleccionado por tiempo de predicción es: Linear Regression


### Decisión del Modelo Final

Ambos modelos con mejor rendimiento, **Random Forest Tuned** y **XGBoost Tuned**, muestran indicios de sobreajuste según nuestra heurística (diferencia entre R² de entrenamiento y prueba > 0.1). 

Al comparar estos dos modelos:
- **Rendimiento (R² de Prueba):** Son prácticamente idénticos (0.805 para RF vs 0.804 para XGB).
- **Grado de Sobreajuste:** La diferencia de R² es ligeramente menor en XGBoost Tuned (0.122) que en Random Forest Tuned (0.143).
- **Tiempos de Ejecución:** XGBoost Tuned es significativamente más rápido tanto en entrenamiento (4.5s vs 16.3s) como en predicción (0.003s vs 0.028s).

El siguiente mejor modelo sin sobreajuste es **XGBoost** (sin tunear), pero su rendimiento es inferior. Dado que **XGBoost Tuned** ofrece un rendimiento casi idéntico al mejor modelo, con un sobreajuste ligeramente menor y tiempos de ejecución muy superiores, se considera la opción más balanceada. Se selecciona **XGBoost Tuned** como el modelo final, asumiendo que el ligero sobreajuste es un compromiso aceptable por el rendimiento y la eficiencia.